In [34]:
from pinecone import Pinecone
from openai import OpenAI
from typing import List
from transformers import pipeline
import sys
sys.path.append('../')
sys.path.extend('../')
from app.settings import inject_settings
from datetime import datetime
import uuid
from vector_db_uploader import embed_text

settings = inject_settings()
pinecone_client = Pinecone(api_key=settings.PINECONE_API_KEY)
index = pinecone_client.Index("salesloft-vista")
client = OpenAI(api_key=settings.OPENAI_API_KEY)
from ai_agent_coach.openai_requester import get_recommendation


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [42]:

# Set transcription parameters
parameters = {
    "model": "gpt-4o-mini",
    "response_format":{ "type": "json_object" },
    "temperature": 0,
    "max_tokens": 1000,
    "top_k":5
}


transcription = "What is the best discount you can give me? Are you flexible?"

In [44]:
recommendation_list = get_recommendation(client, index, parameters, transcription)


In [45]:
recommendation_list

["Ask about their budget to explore flexibility: 'What does your budget look like for this initiative?'",
 "Mention payment options: 'We offer installment plans to make budgeting easier.'"]

In [46]:
[print(f"\t* {rec}") for rec in recommendation_list]


	* Ask about their budget to explore flexibility: 'What does your budget look like for this initiative?'
	* Mention payment options: 'We offer installment plans to make budgeting easier.'


[None, None]

In [29]:

query_string = "What is the best discount you can give me? Are you flexible?"

query_vector = embed_text(client=client, text=query_string)
document_payload_response = index.query(
                                        vector = query_vector,
                                        top_k=5,
                                        include_metadata=True
                                    )
text_chunks = [match.metadata for match in document_payload_response['matches']]
documents = [doc['textchunk'] for doc in text_chunks]

In [30]:
# contatenated_string_list = []

# for idx, doc in enumerate(documents, start =1):
#     contatenated_string_list.append(f"Context document #{idx}: {doc}\n")

# concatenated_document_string ="\n".join(contatenated_string_list)

In [31]:
def concatenate_documents(documents):
    """
    Concatenate a list of documents into a single string with each document
    prefixed by its index.

    Args:
        documents (list of str): A list of document strings to be concatenated.

    Returns:
        str: A single string where each document is prefixed by its index and separated by newlines.
    """
    concatenated_string_list = []

    # Iterate over the documents with an index starting from 1
    for idx, doc in enumerate(documents, start=1):
        # Append the formatted document string to the list
        concatenated_string_list.append(f"Context document #{idx}: {doc}\n")
    
    # Join all document strings into a single string with newlines
    concatenated_document_string = "\n".join(concatenated_string_list)
    
    return concatenated_document_string


In [32]:
concatenated_doc_context = concatenate_documents(documents)

In [33]:
print(concatenated_doc_context)

Context document #1: your budget look like for this initiative? Are there areas where you see flexibility?" 4. Position Pricing as Flexible: "We offer several packages to fit different needs and budgets. Our standard 

Context document #2: see if there’s flexibility in their budget. "If we can show a strong ROI, would there be room to adjust the budget slightly?" 4. Position Pricing as Flexible Offer Pricing Options - Multiple Tiers or 

Context document #3: Installment Plans: If available, discuss installment payment options. "We offer a 12-month installment plan which can make budgeting easier for you." - Deferred Payments: If applicable, offer 

Context document #4: open-ended questions to understand their budget constraints without making assumptions. "Can you share with me what your budget looks like for this project?" - Explore Financial Flexibility: Probe to 

Context document #5: customizing a package to better fit their needs. "We can also tailor a package that fits your speci

In [58]:
def read_file(file_path: str) -> str:
    with open(file_path, "r", encoding='utf-8') as file:
        return file.read()
    
def chunk_text(text: str, chunk_size: int) -> list:
    chunks = []
    curr_chunk = ""

    for word in text.split():
        if len(curr_chunk) + len(word) < chunk_size:
            curr_chunk += word + " "
        else:
            chunks.append(curr_chunk)
            curr_chunk = word + " "

    chunks.append(curr_chunk)
    return chunks

def flatten_list_comprehension(nested_list):
    return [item for sublist in nested_list for item in sublist]

In [53]:
import os


files = os.listdir()
text_files = [file for file in files if '.txt' in file]

list_of_transcripts = [read_file(file) for file in text_files]
list_of_chunks = [chunk_text(transcript, 200) for transcript in list_of_transcripts]
flat_chunk_list = flatten_list_comprehension(list_of_chunks)


In [57]:
len(list_of_chunks[2])

26

In [59]:
flat_chunk_list = flatten_list_comprehension(list_of_chunks)

In [60]:
len(flat_chunk_list)

79

In [61]:
flat_chunk_list

['\ufeff1. Deep Understanding of Buyer’s Needs Comprehensive Needs Assessment - Ask Open-Ended Questions: Begin by understanding the buyer’s specific needs, challenges, and goals. "Can you share with me the ',
 'main challenges you\'re facing with your current solution?" - Tailor Your Approach: Customize your value proposition based on the buyer’s responses. "I understand that improving efficiency is a top ',
 'priority for you. Our solution is designed to streamline your operations, saving you both time and money." Confirm Understanding - Summarize Buyer’s Needs: Restate the buyer’s key challenges and ',
 'goals to confirm your understanding. "So, reducing operational costs and enhancing productivity are your primary objectives, correct?" - Seek Confirmation: Ensure the buyer agrees with your summary. ',
 '"Did I capture that correctly? Is there anything else you’d like to add?" 2. Articulate Clear Value Proposition Highlight Key Benefits - Specific Benefits: Clearly outline the benef

In [62]:
os.getcwd()

'/Users/carlos.salas/Documents/sl-vista-backend/devtools'